In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
townships = pd.read_csv('../data/processed/edgeprop_townships_preprocessed.csv')
townships['coordinates'] = list(zip(townships['latitude'], townships['longitude']))
townships

,Unnamed: 0,project_id,latitude,longitude,state,area,median_psf,median_price,poi_nearby_education,poi_nearby_transportation,poi_nearby_healthcare,nearest_school,nearest_train_station,nearest_hospital,total_poi_nearby,nearest_poi,transaction_count,coordinates
0,1,28054,1.850838,103.098501,Johor,Parit Raja,32.5,140000.0,7,0,3,698.877585,118249.246278,16451.842589,10,698.877585,4,"(1.8508384999999998, 103.0985005)"
1,3,19000,2.424192,103.837082,Johor,Mersing,92.5,290000.0,10,0,2,583.035476,179112.693473,1101.439888,12,583.035476,4,"(2.4241923, 103.8370822)"
2,4,19009,2.502480,102.831990,Johor,Segamat,56.0,160000.0,16,0,4,479.442036,67412.610445,1880.370354,20,479.442036,4,"(2.50248, 102.83199)"
3,5,19036,1.989595,102.873294,Johor,Parit Sulong,122.5,324400.0,7,0,2,642.169632,88940.537416,15543.042648,9,642.169632,4,"(1.9895953000000002, 102.8732944)"
4,9,28046,2.109806,103.396274,Johor,Kluang,102.0,235000.0,4,0,1,377.403414,135795.961947,10312.273554,5,377.403414,4,"(2.1098055000000002, 103.39627409999999)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6757,7466,23082,3.016821,101.623228,Selangor,Bandar Puteri Puchong,542.0,965000.0,19,4,3,851.781025,1181.540717,777.226157,26,777.226157,193,"(3.0168207000000002, 101.6232283)"
6758,7467,11372,3.053950,101.528999,Selangor,Shah Alam,271.0,200000.0,21,2,3,330.456470,492.440153,1366.133911,26,330.456470,193,"(3.0539500713347993, 101.52899932861)"
6759,7468,11479,3.130250,101.473999,Selangor,Setia Alam/Alam Nusantara,NaN,NaN,8,0,0,1549.983563,7098.462905,5893.744567,8,1549.983563,0,"(3.13025, 101.47399899999999)"
6760,7469,10419,2.876342,101.806980,Selangor,Bangi,246.0,382500.0,6,0,1,738.335774,3857.170753,8069.968183,7,738.335774,188,"(2.8763422999999997, 101.80697959999999)"


In [3]:
from geopy.distance import distance

# calculate distance between two points
def calc_distance(source, target):
    return(distance(source, target).m)


# find nearby city
def nearby_city(target_coordinate):
    townships = pd.read_csv('../data/processed/edgeprop_townships_preprocessed.csv')
    townships['coordinates'] = list(zip(townships['latitude'], townships['longitude']))
    
    townships['distance'] = townships['coordinates'].apply(
        lambda x: calc_distance(x, target_coordinate)
    )

    # return all cities within 15km
    nearby_cities = townships[townships['distance'] <= 5000][
        ['area', 'distance']
    ].groupby('area').min().reset_index().sort_values('distance')

    return (json.dumps(list(nearby_cities['area'])))




In [4]:
nearby_cities = nearby_city((3.064520, 101.665436))

In [5]:
nearby_cities

'["Bukit Jalil", "Bandar Baru Sri Petaling", "Jalan Klang Lama (Old Klang Road)", "Taman OUG", "Bandar Kinrara Puchong", "Petaling Jaya", "Kuchai Lama", "Kelana Jaya", "Salak Selatan", "Sri Petaling", "Puchong", "Pantai Dalam/Kerinchi", "Seri Kembangan", "Puncak Jalil", "Taman Desa", "Sungai Besi", "Desa Petaling", "Bangsar South", "Bandar Tasik Selatan"]'

In [6]:
from geopy.distance import distance

# calculate distance between two points
def calc_distance(source, target):
    return(distance(source, target).m)


# find nearby city
def nearest_poi(target_coordinate):
    # target_coordinate = (request.args.get('lat'), request.args.get('lng'))

    pois = pd.read_csv('../data/processed/iproperty_pois_preprocessed.csv')
    pois['coordinates'] = list(zip(pois['latitude'], pois['longitude']))
    
    pois['distance'] = pois['coordinates'].apply(
        lambda x: calc_distance(x, target_coordinate)
    )

    # return all cities within 15km
#     nearby_cities = townships[townships['distance'] <= 5000][
#         ['area', 'distance']
#     ].groupby('area').min().reset_index().sort_values('distance')

#     return (json.dumps(list(nearby_cities['area'])))


    # return all cities within 15km
    return(pois['distance'].min())

#     return(json.dumps(list(nearest_poi)))




In [7]:
nearest_poi((3.064520, 101.665436))

156.40462573036172

In [8]:
transactions = pd.read_csv('../data/processed/edgeprop_transactions_processed.csv')

In [13]:
transactions['property_type'].unique()

array(['Flat', 'Condominium/Apartment', 'Town House', 'Terrace House',
       'Hotel/Service Apartment', 'Semi-Detached House', 'Detached House',
       'Low-Cost House', 'Cluster House'], dtype=object)

In [14]:
transactions['tenure'].unique()

array(['Freehold', 'Leasehold'], dtype=object)

In [17]:
import pickle
model = pickle.load(open('../data/model.pkl', 'rb'))


In [49]:
input_features = pd.DataFrame([
    json.loads('{
       "nearest_poi": "305.7811181613881", "city": "Petaling Jaya", 
       "property_type": "Detached House", "tenure": "Freehold", 
       "area_sqft": "1170", "bedrooms": "3"
   }')
])
# predicted_price = model.predict(input_features)

In [50]:
input_features

,nearest_poi,city,property_type,tenure,area_sqft,bedrooms
0,305.7811181613881,Petaling Jaya,Detached House,Freehold,1170,3


In [51]:
predicted_price = model.predict(input_features)

/home/kitlim/py3_kernel/lib/python3.6/site-packages/sklearn/compose/_column_transformer.py:440: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


In [53]:
predicted_price[0]

246838.0799845625